# Fit a FRED


A FRED (fast rise, exponential decay) pulse is a common model for glitches in LISA data. 
Here we show how to fit a FRED pulse to some synthetic data. 

We use the following parameterisation of the FRED equation:

$ S(t|A,\Delta,\tau,\xi) = A \exp \left[ - \xi \left(
        \frac{t - \Delta}{\tau} + \frac{\tau}{t-\Delta}  \right) -2  \right]$



Where 
- $A$ (the scale) multiplies the entire expression, controlling the overall amplitude, 
- $\Delta$ (the start) is the time at which the pulse begins, and
- $\tau$ is the duration, and
- $\xi$ is the asymmetry parameter 


The exponential term $\exp(...)$ creates the pulse shape:
- $-\xi$` in front of the parentheses controls the overall decay rate and asymmetry.
- $(\tau / (t - \Delta))$ contributes to the initial rise of the pulse.
- $((t - \Delta) / \tau)$ contributes to the later decay of the pulse.
- The -2.0 term helps normalize the pulse shape.

The interplay between $\tau$ and $\xi$ determines the specific shape of the pulse:
- Larger $\tau$ values result in a wider pulse.
- Larger $\xi$ values make the pulse more asymmetric, with a faster rise and slower decay.

<video src="_static/FREDPulseAnimation.mp4"></video>


We will generate some synthetic data with a FRED pulse and then fit a FRED pulse to the data.

## SNR ~ 1000

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np

from lisa_glitch_buster.backend.model import FRED_pulse
from lisa_glitch_buster.glitch_fitter_bilby import GlitchFitter

np.random.seed(0)

T_START, T_END = 0, 10
OUTDIR = "outdir_glitch_pe"

def generate_dataset(N, injection_params):
    times = np.linspace(T_START, T_END, N)
    # white noise
    noise = np.random.normal(0, injection_params['sigma'], N)
    # inject a FRED pulse
    pulse = FRED_pulse(times, **injection_params)
    data = pulse + noise
    return data, times


true_params = {
    "start": 2,
    "scale": 100,
    "tau": 1,
    "xi": 1,
    "sigma": 1
}

data, times = generate_dataset(1000, true_params)


fitter = GlitchFitter(
    data=data,
    times=times,
    trigger_time=true_params["start"],
    model="FRED_pulse",
    injection_parameters=true_params,
    outdir=OUTDIR,
)
_ = fitter.plot(save_fn=f"{OUTDIR}/data.png")


![](outdir_glitch_pe/data.png)

## SNR ~ 20

In [ ]:
res = fitter.run_sampler(
    plot=True,
    sampler="dynesty",
    clean=True,
)


![](outdir_glitch_pe/glitch_fit_corner.png)
```

In [ ]:
true_params = {
    "start": 2,
    "scale": 2,
    "tau": 1,
    "xi": 1,
    "sigma": 0.5
}

data, times = generate_dataset(1000, true_params)

OUTDIR = "outdir_glitch_pe_small"
fitter = GlitchFitter(
    data=data,
    times=times,
    trigger_time=true_params["start"],
    model="FRED_pulse",
    injection_parameters=true_params,
    outdir=OUTDIR,
)
res = fitter.run_sampler(
    plot=True,
    sampler="dynesty",
    clean=True,
)



In [ ]:
ax = fitter.plot()
ax.get_figure().savefig(f"{OUTDIR}/glitch_fit.png")
fitter.plot_corner(f"{OUTDIR}/corner.png")

![](outdir_glitch_pe_small/glitch_fit.png)
![](outdir_glitch_pe_small/corner.png)